In [ ]:
import xarray as xr
import numpy as np
import rioxarray
import h5netcdf
import h5py
import netCDF4
import scipy.ndimage

import hvplot.xarray
import geoviews as gv

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
surface_velocity = xr.open_dataset('data/GRE_G0120_0000.nc')
surface_velocity_spacing_m = np.median(np.diff(surface_velocity['x']))
assert(np.max(np.diff(surface_velocity['x'])) == surface_velocity_spacing_m)
assert(np.min(np.diff(surface_velocity['x'])) == surface_velocity_spacing_m)

age_model = xr.open_dataset('data/RRRAG4_Greenland_1993_2013_01_age_grid.nc',
                               engine='h5netcdf').rename(
                                   {'number of grid points in x-direction': 'x', 'number of grid points in y-direction': 'y', 'number of vertical layers': 'layers', 'number of isochrones': 'original_isochrones',
                                    'x': 'x_km', 'y': 'y_km'}
                                   )
# Convert km to meters for consistency with the rest of the world
age_model = age_model.assign_coords({'x':age_model.x_km[:,0]*1000, 'y':age_model.y_km[0,:]*1000})

bedmachine = xr.open_dataset('data/BedMachineGreenland-v5.nc')

In [ ]:
crs_3413 = ccrs.Stereographic(central_latitude=90, central_longitude=-45, true_scale_latitude=70)

In [ ]:
isochrone_ages = np.logspace(3, 6, 20)

def interpolate_age_depths(age_norm, age_norm_depths = age_model['depth_norm'].to_numpy(), query_ages = isochrone_ages):
    return np.interp(query_ages, age_norm, age_norm_depths, left=np.nan, right=np.nan)

isochrones_norm = xr.apply_ufunc(interpolate_age_depths, age_model['age_norm'], input_core_dims=[['layers']], output_core_dims=[['isochrone']], vectorize=True)
isochrones_norm = isochrones_norm.assign_coords({'isochrone': isochrone_ages})

In [ ]:
smoothing_kernel_std = 5000 # m

In [ ]:
# isochrones_norm is in normalized depth (0 == surface, 1 == bed)
# want to convert this into elevation relative to the geoid in order to find layer geometry and then slope
layer_elevation = ((1 - isochrones_norm) * age_model['thick']) + bedmachine['bed'].interp(x=age_model['x'], y=age_model['y'])
layer_elevation.name = "Layer Elevation"

layer_elevation_spacing_m = np.median(np.diff(layer_elevation['x']))
assert(np.max(np.diff(layer_elevation['x'])) == layer_elevation_spacing_m)
assert(np.min(np.diff(layer_elevation['x'])) == layer_elevation_spacing_m)
assert(np.max(np.diff(layer_elevation['y'])) == layer_elevation_spacing_m)
assert(np.min(np.diff(layer_elevation['y'])) == layer_elevation_spacing_m)

layer_elevation_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, layer_elevation, input_core_dims=[['x', 'y']], output_core_dims=[['x', 'y']], vectorize=True, kwargs={'sigma': smoothing_kernel_std / layer_elevation_spacing_m})

In [ ]:
surf_u_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, surface_velocity['vx'], kwargs={'sigma': smoothing_kernel_std / surface_velocity_spacing_m, 'truncate': 2})
surf_v_filt = xr.apply_ufunc(scipy.ndimage.gaussian_filter, surface_velocity['vx'], kwargs={'sigma': smoothing_kernel_std / surface_velocity_spacing_m, 'truncate': 2})

In [ ]:
layer_dx_dz = layer_elevation_filt.differentiate("x").differentiate("isochrone") / layer_elevation.differentiate("isochrone")
layer_dy_dz = layer_elevation_filt.differentiate("y").differentiate("isochrone") / layer_elevation.differentiate("isochrone")

us = surf_u_filt.interp(x=age_model['x'], y=age_model['y']).squeeze() # Surface velocity X component
vs = surf_v_filt.interp(x=age_model['x'], y=age_model['y']).squeeze() # Surface velocity Y component
us_dx = us.differentiate("x")
vs_dy = vs.differentiate("y")

#stab_crit = (us * layer_dx_dz) + (vs * layer_dy_dz)
#stab_crit = us_dx + vs_dy
stab_crit = (us * layer_dx_dz) + (vs * layer_dy_dz) + us_dx + vs_dy

p = stab_crit.hvplot.image('x', 'y', crs=crs_3413, coastline=True, cmap='bwr', frame_height=500, clim=(-1e-3,1e-3), rasterize=False, dynamic=True)
#stab_crit.hvplot.quadmesh('x', 'y', crs=crs_3413, coastline=True, groupby=['isochrone'], cmap='RdBu', clim=(-1e-6,1e-6), frame_height=1000, rasterize=True)


#layer_dx_dz.hvplot.quadmesh('x', 'y', crs=crs_3413, rasterize=True, coastline=True, frame_height=500, groupby=['isochrone'])

#hvplot.save(p, 'surf_vel_divergence.png')
p

In [ ]:
surf_vel_divergence = us_dx + vs_dy
p = surf_vel_divergence.hvplot.image('x', 'y', crs=crs_3413, coastline=True, cmap='bwr', frame_height=500, clim=(-1e-3,1e-3), rasterize=False)
#hvplot.save(p, f'surf_vel_divergence_smoothstd{smoothing_kernel_std}.png')
p

In [ ]:
layer_slopes = np.sqrt( (layer_elevation_filt.differentiate("x") ** 2) + (layer_elevation_filt.differentiate("y") ** 2) )
p = (layer_slopes * 1000).hvplot.image('x', 'y', crs=crs_3413, coastline=True, frame_height=500, clim=(0,20), rasterize=False)
p.opts(title=f"Layer slope [m/km]")

In [ ]:
qts = np.arange(0.1, 0.9, 0.01)

layer_geom_terms = (us * layer_dx_dz) + (vs * layer_dy_dz)
surf_vel_divergence = us_dx + vs_dy

p = layer_geom_terms.quantile(qts).rename('(us * layer_dx_dz) + (vs * layer_dy_dz)').hvplot(label='(us * layer_dx_dz) + (vs * layer_dy_dz)') * \
    surf_vel_divergence.quantile(qts).rename('us_dx + vs_dy').hvplot(label='us_dx + vs_dy').opts(xlabel='') * \
    (layer_geom_terms + surf_vel_divergence).quantile(qts).rename('sum').hvplot(label='sum').opts(xlabel='')
p.opts(ylabel='', xlabel='quantile', legend_position="top_left", show_grid=True, ylim=(-5e-4, 1e-3), title=f'Smoothing Std {smoothing_kernel_std/1000} km')
#hvplot.save(p, f'stability_terms_comparison_smoothstd{smoothing_kernel_std}.png')
p

In [ ]:
abs_ratio = np.abs(surf_vel_divergence / layer_geom_terms)
p = abs_ratio.quantile(qts).rename('quantiles').hvplot(label='abs(us_dx + vs_dy) / abs((us * layer_dx_dz) + (vs * layer_dy_dz))').opts(xlabel='')
p.opts(ylabel='Ratio of surface velocity divergence to\nlayer geometry terms', xlabel='quantile', show_grid=True, ylim=(0, 10))
#p.opts(ylabel='', xlabel='quantile', legend_position="top_left", show_grid=True, ylim=(0, 5), title=f'Smoothing Std {smoothing_kernel_std/1000} km')

## OLD STUFF

In [ ]:
layer_idx = int(np.abs(age_model['depth_norm'] - 0.5).argmin())
fractional_thickness_of_layer = float(age_model['depth_norm'][layer_idx])
age_at_layer = age_model['age_norm'].sel(layers=layer_idx)
age_at_layer.hvplot.image('x', 'y', crs=crs_3413, coastline=True, rasterize=False)

In [ ]:
layer_dx_at_sel_layer = layer_elevation_filt.differentiate('x').interp(isochrone=age_at_layer)
layer_dy_at_sel_layer = layer_elevation_filt.differentiate('y').interp(isochrone=age_at_layer)

layer_slope_at_sel_layer = np.sqrt(layer_dx_at_sel_layer**2 + layer_dy_at_sel_layer**2)


p = (layer_slope_at_sel_layer * 1000).hvplot.image('x', 'y', crs=crs_3413, coastline=True, rasterize=False, frame_height=500, clim=(0,20))
p.opts(title=f"Layer slope at {fractional_thickness_of_layer*100:.0f}% of ice thickness [m/km]")

In [ ]:
surface_velocity

In [ ]:
layer_induced_apparent_vertical_motion = (layer_dx_at_sel_layer * us) + (layer_dy_at_sel_layer * vs) 

p = layer_induced_apparent_vertical_motion.hvplot.image('x', 'y', crs=crs_3413, coastline=True, rasterize=False, cmap='bwr', frame_height=500, clim=(-0.5,0.5))
p.opts(title=f"Layer induced apparent vertical motion at\n{fractional_thickness_of_layer*100:.0f}% of ice thickness [m/yr]")

In [ ]:
surf_vel_mag = np.sqrt(us**2 + vs**2)
p = surf_vel_mag.hvplot.image('x', 'y', crs=crs_3413, coastline=True, rasterize=False, frame_height=500, clim=(0,100), cmap='OrRd')
p.opts(title=f"Surface velocity [m/yr]")

In [ ]:
# p = us.quantile(qts).hvplot(label='us') * \
#     vs.quantile(qts).hvplot(label='vy')
# p.opts(ylabel='', legend_position="top_left")
# #hvplot.save(p, 'surface_velocity.png')

In [ ]:
# t = layer_dx_dz
# print((t.mean(), t.std()))
# print(abs(t).mean())
# print(t.quantile([0.01, 0.1, 0.5, 0.9, 0.99]))
# t.hvplot.hist(bins=500)

In [ ]:
stab_crit

In [ ]:
layer_elevation.hvplot.quadmesh('x', 'y', crs=crs_3413, rasterize=True, coastline=True, frame_height=500, groupby=['isochrone'])
# TODO : haven't sanity checked this. see if it makes sense. Then work on layer slopes estimates

In [ ]:
isochrones_norm.hvplot.quadmesh('x', 'y', crs=crs_3413, rasterize=True, coastline=True, height=500, groupby=['isochrone'])

In [ ]:
#isochrones_norm.to_dataset().to(hv.quadmesh, 'x', 'y', groupby=['isochrone'], crs=crs_3413, rasterize=True, coastline=True)
isochrones_norm.assign_coords({'isochrone': [1,2,3,4]}).hvplot.quadmesh('x', 'y', crs=crs_3413, rasterize=True, coastline=True, groupby=['isochrone'])

In [ ]:
# Giving up on matplotlib since I got HV working

# #epsg_3413_maybe = ccrs.NorthPolarStereo(true_scale_latitude=70, central_longitude=70)
# crs_3413 = ccrs.Stereographic(central_latitude=90, central_longitude=-45, true_scale_latitude=70) # I think this is right, but the rest of this still doesn't work
# fig, ax = plt.subplots(subplot_kw={'projection': crs_3413})

# ax.coastlines()
# #ax.set_xlim(-6e6, 9e6)
# #ax.set_ylim(-3e7, -7e6)
# #ax.set_aspect('equal')
# isochrones_norm.loc[{'isochrone': 1}].plot(ax=ax, transform=crs_3413)

In [ ]:
layer_idx = 0
normalized_layer_depth = age_model['age_norm'].loc['number of vertical layers' == layer_idx]
# Note: bedmachine x and y units are EPSG:3413 in meters, age_model x and y units are EPSG:3413 in km
layer_elevation = (normalized_layer_depth * age_model['thick']) #+ bedmachine['bed'].interp(x=age_model['x']*1e3, y=age_model['y']*1e3)


In [ ]:
normalized_layer_depth.plot()

In [ ]:
age_model

In [ ]:
age_model['age_norm']

In [ ]:
netCDF4.Dataset('data/RRRAG4_Greenland_1993_2013_01_age_grid.nc', 'r').variables['age_iso'][:]

In [ ]:
# Open 'data/RRRAG4_Greenland_1993_2013_01_radiostratigraphy.mat' with h5py
f = h5py.File('data/RRRAG4_Greenland_1993_2013_01_radiostratigraphy.mat', 'r')
list(f['gris_strat']['campaign']['name'][5])

In [ ]:
xr.open_dataset(xr.backends.H5NetCDFStore(f['gris_strat']))

In [ ]:
xr.Dataset(f['gris_strat'])